
# 作業 : (Kaggle)鐵達尼生存預測¶

    分數以網站評分結果為準, 請同學實際將提交檔(*.csv)上傳試試看
    https://www.kaggle.com/c/titanic/submit

# [作業目標]

    試著模仿範例寫法, 在鐵達尼生存預測中, 觀察觀查混合泛化 (Blending) 的寫法與效果

# [作業重點]

    觀察混合泛化的準確度 (In[14]), 是否比單一模型準確度為高 (In[11~13])
    除了我們的權重, 同學也可以試著自行調整權重 (注意:權重和=1), 看看有什麼影響
    Hint : 除了權重, 分類預測的調整, 還可以調整什麼地方?



In [10]:
import pandas as pd
import numpy as np
import copy, time
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV , cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, Imputer
import warnings
warnings.filterwarnings('ignore')


data_path ='E:\\python\\ML-day100\\data\\'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')


train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
# 檢查空缺值的狀態
def nan_check(df_data):
    data_na = (df_data.isnull().sum() / len(df_data)) * 100
    data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})
    display(missing_data.head(10))
nan_check(df)

,Missing Ratio
Cabin,77.463713
Age,20.091673
Embarked,0.152788
Fare,0.076394


In [12]:
df["Sex"] = df["Sex"].map({"male": 0, "female":1})
df["Fare"] = df["Fare"].map(lambda i: np.log(i) if i > 0 else 0)
df["Age"] = df["Age"].fillna(df['Age'].median())

In [4]:
#Title做分類
df_title = [i.split(",")[1].split(".")[0].strip() for i in df["Name"]]
df["Title"] = pd.Series(df_title)
df["Title"] = df["Title"].replace(['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df["Title"] = df["Title"].map({"Master":0, "Miss":1, "Ms" : 1 , "Mme":1, "Mlle":1, "Mrs":1, "Mr":2, "Rare":3})
df["Title"] = df["Title"].astype(int)
df = pd.get_dummies(df, columns = ["Title"])

df["Fsize"] = df["SibSp"] + df["Parch"] + 1
df['Single'] = df['Fsize'].map(lambda s: 1 if s == 1 else 0)
df['SmallF'] = df['Fsize'].map(lambda s: 1 if  s == 2  else 0)
df['MedF'] = df['Fsize'].map(lambda s: 1 if 3 <= s <= 4 else 0)
df['LargeF'] = df['Fsize'].map(lambda s: 1 if s >= 5 else 0)

In [13]:
# Ticket分類
Ticket = []
for i in list(df.Ticket):
    if not i.isdigit() :
        Ticket.append(i.replace(".","").replace("/","").strip().split(' ')[0])
    else:
        Ticket.append("X")        
df["Ticket"] = Ticket
df = pd.get_dummies(df, columns = ["Ticket"], prefix="T")

#Cabin分類
df["Cabin"] = pd.Series([i[0] if not pd.isnull(i) else 'X' for i in df['Cabin'] ])
df = pd.get_dummies(df, columns = ["Cabin"], prefix="Cabin")

# Embarked, Pclass分類
df = pd.get_dummies(df, columns = ["Embarked"], prefix="Em")
df["Pclass"] = df["Pclass"].astype("category")
df = pd.get_dummies(df, columns = ["Pclass"], prefix="Pc")

In [14]:
# 捨棄 Name
df.drop(labels = ["Name"], axis = 1, inplace = True)

In [15]:
nan_check(df)

,Missing Ratio


In [16]:
df.head()

,Sex,Age,SibSp,Parch,Fare,T_A,T_A4,T_A5,T_AQ3,T_AQ4,...,Cabin_F,Cabin_G,Cabin_T,Cabin_X,Em_C,Em_Q,Em_S,Pc_1,Pc_2,Pc_3
0,0,22.0,1,0,1.981001,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,1
1,1,38.0,1,0,4.266662,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,1,26.0,0,0,2.070022,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
3,1,35.0,1,0,3.972177,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
4,0,35.0,0,0,2.085672,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1


In [17]:
df = MinMaxScaler().fit_transform(df)
train_num = train_Y.shape[0]
train_X = df[:train_num]
test_X = df[train_num:]

In [19]:
# 線性迴歸預測檔
LR = LogisticRegression()
parameter_LR = dict()
grid_search_LR = GridSearchCV(LR, dict(),scoring='accuracy',cv=10).fit(train_X , train_Y)
LR = grid_search_LR.best_estimator_ 
LR.fit(train_X, train_Y)
LR_pred = LR.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'PassengerId': ids, 'Survived': LR_pred})
sub['Survived'] = sub['Survived'].map(lambda x:1 if x>0.5 else 0) 
sub.to_csv('Day_049_submit_titanic_LR.csv', index=False)
print('Best estimator LogisticRegression:',grid_search_LR.best_estimator_  )

Best estimator LogisticRegression: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)


In [20]:
# 梯度提升機
gdbt = GradientBoostingClassifier()
parameter_gdbt = dict()
grid_search_gdbt = GridSearchCV(gdbt,  dict(),scoring='accuracy',cv=10).fit(train_X , train_Y)
gdbt = grid_search_gdbt.best_estimator_
gdbt.fit(train_X, train_Y)
gdbt_pred = gdbt.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'PassengerId': ids, 'Survived': gdbt_pred})
sub['Survived'] = sub['Survived'].map(lambda x:1 if x>0.5 else 0) 
sub.to_csv('Day_049_submit_titanic_gdbt.csv', index=False)
print('Best estimator GradientBoostingClassifier:',grid_search_gdbt.best_estimator_  )

Best estimator GradientBoostingClassifier: GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)


In [21]:
# 隨機森林
RFC = RandomForestClassifier()
parameter_RFC = dict()
grid_search_RFC = GridSearchCV(RFC,  dict(),scoring='accuracy',cv=10).fit(train_X , train_Y)
RFC = grid_search_RFC.best_estimator_
RFC.fit(train_X, train_Y)
RFC_pred = RFC.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'PassengerId': ids, 'Survived': RFC_pred})
sub['Survived'] = sub['Survived'].map(lambda x:1 if x>0.5 else 0) 
sub.to_csv('Day_049_submit_titanic_rf.csv', index=False)
print('Best estimator RandomForestClassifier:',grid_search_RFC.best_estimator_  )

Best estimator RandomForestClassifier: RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


# 作業

    雖然同樣是混合泛化，分類預測其實與回歸預測有相當多的差異性， 因為鐵達尼預測的結果是 '生存/死亡'，輸出不是 0 就是 1
    因此要用權重混合時，需要以以機率的形式混合，因此我們在作業前幾格當中，先幫各位同學把預測值寫成了機率的形式
    (請同學把下列程式完成，並將結果提交到 Kaggle 網站看看結果)

    但是光是這樣，分類問題的混合泛化就能比單模預測還要好嗎?
    已經快要期中考了，這裡請同學挑戰看看，還有沒有什麼方法可以改進混合泛化的結果?

In [23]:
# 作業


# LR: 0.66985
# gdbt: 0.60287
# RFC: 0.698561

blending_pred = LR_pred*0.4  + gdbt_pred*0.2 + RFC_pred*0.4
sub = pd.DataFrame({'PassengerId': ids, 'Survived': blending_pred})
sub['Survived'] =  sub['Survived'].map(lambda x:1 if x>0.7 else 0) 
sub.to_csv('Day_049_submit_titanic_blending.csv', index=False)
#Score=0.68899